In [1]:
import mediapipe as mp
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distance(p1, p2):
    return ((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2 + (p2.z - p1.z) ** 2) ** 0.5

In [4]:
def finger_raised(landmarks, finger_tip, finger_dip, finger_pip, finger_mcp, umbral=0.2):
    return ( 
            landmarks[finger_tip].y < landmarks[finger_dip].y and
            landmarks[finger_dip].y < landmarks[finger_pip].y and
            landmarks[finger_pip].y < landmarks[finger_mcp].y
    )

In [5]:
cap = cv2.VideoCapture(0)

In [ ]:
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence = 0.5) as hands:
   
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Flip the image horizontally for a later selfie-view display
    image = cv2.flip(image, 1)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        # Draw hand landmarks
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        landmarks = hand_landmarks.landmark
        
        PULGAR_TIP = 4
        INDICE_TIP = 8
        MEDIO_TIP = 12
        ANULAR_TIP = 16
        MENIQUE_TIP = 20

        fingers = {
                'pulgar': (PULGAR_TIP, 2, 1, 0),
                'indice': (INDICE_TIP, 6, 5, 0),
                'medio': (MEDIO_TIP, 10, 9, 0),
                'anular': (ANULAR_TIP, 14, 13, 0),
                'menique': (MENIQUE_TIP, 18, 17, 0)
            }
        
        raised_fingers = []
        for name, (tip, dip, pip, mcp) in fingers.items():
            if finger_raised(landmarks, tip, dip, pip, mcp):
                raised_fingers.append(name)

        cv2.putText(image, f'Fingers: {", ".join(raised_fingers)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Tracking', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()